# 현재 코드는 4개의 대분류 클레스에 대해 평균값이 어떻게 되는지 확인하고자 한다.

In [1]:
# Cell 1: 필요한 라이브러리 임포트
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import os
import json
from pathlib import Path

In [2]:
# Cell 2: 카테고리 매핑 정의
category_mapping = {
    # 대형폐기물 (Large Waste Items)
    'arcade machine': 'Large Waste Items',
    'Audio': 'Large Waste Items',
    'Computer': 'Large Waste Items',
    'fax machine': 'Large Waste Items',
    'Main unit': 'Large Waste Items',
    'Monitor': 'Large Waste Items',
    'Printer': 'Large Waste Items',
    'sewing machine': 'Large Waste Items',
    'Speaker': 'Large Waste Items',
    'typewriter': 'Large Waste Items',
    'Vacuum cleaner': 'Large Waste Items',
    'Video player': 'Large Waste Items',
    'Bathtub': 'Large Waste Items',
    'Sink': 'Large Waste Items',
    'Kitchen sink': 'Large Waste Items',
    'Toilet bowl': 'Large Waste Items',
    'Bed': 'Large Waste Items',
    'Bookcase': 'Large Waste Items',
    'Bookstand': 'Large Waste Items',
    'Cabinet': 'Large Waste Items',
    'chair': 'Large Waste Items',
    'Cupboard': 'Large Waste Items',
    'Desk': 'Large Waste Items',
    'Dining table': 'Large Waste Items',
    'Display cabinet': 'Large Waste Items',
    'Display stand': 'Large Waste Items',
    'Drawer unit': 'Large Waste Items',
    'Shoe rack': 'Large Waste Items',
    'Small cabinet': 'Large Waste Items',
    'Sofa': 'Large Waste Items',
    'Table': 'Large Waste Items',
    'TV stand': 'Large Waste Items',
    'Vanity table': 'Large Waste Items',
    'Wardrobe': 'Large Waste Items',
    'Air conditioner': 'Large Waste Items',
    'Air purifier': 'Large Waste Items',
    'dish dryer': 'Large Waste Items',
    'Electric rice cooker': 'Large Waste Items',
    'Fan': 'Large Waste Items',
    'Gas oven range': 'Large Waste Items',
    'Heater': 'Large Waste Items',
    'Humidifier': 'Large Waste Items',
    'Microwave': 'Large Waste Items',
    'refrigerator': 'Large Waste Items',
    'Spin dryer': 'Large Waste Items',
    'TV': 'Large Waste Items',
    'Washing machine': 'Large Waste Items',
    'Aquarium': 'Large Waste Items',
    'Bamboo mat': 'Large Waste Items',
    'Bedding items': 'Large Waste Items',
    'bicycle': 'Large Waste Items',
    'Carpet': 'Large Waste Items',
    'Clothes drying rack': 'Large Waste Items',
    'Coat rack': 'Large Waste Items',
    'Door panel': 'Large Waste Items',
    'Earthenware jar': 'Large Waste Items',
    'Floor covering': 'Large Waste Items',
    'Frame': 'Large Waste Items',
    'lumber': 'Large Waste Items',
    'Mannequin': 'Large Waste Items',
    'Mat': 'Large Waste Items',
    'Piano': 'Large Waste Items',
    'Rice storage container': 'Large Waste Items',
    'Signboard': 'Large Waste Items',
    'Stroller': 'Large Waste Items',
    'Wall clock': 'Large Waste Items',
    'Water tank': 'Large Waste Items',
    'audio cabinet': 'Large Waste Items',
    'suitcase': 'Large Waste Items',
    
    # 기타 카테고리
    'PP bag': 'PP bag',
    'General waste bag': 'General Waste',
    'waste pile': 'General Waste',
    'CleanNet': 'CleanNet'
}

In [3]:
# Cell 3: 폴더 내 모든 이미지 객체 감지용 함수
def detect_objects(model_path, image_dir):
    # 모델 로드
    model = YOLO(model_path)
    
    # 출력 디렉토리 생성
    output_dir = './runs/detect/predict/'
    os.makedirs(output_dir, exist_ok=True)
    
    # 모든 이미지 파일 가져오기 (jpg, jpeg, png)
    image_paths = []
    for ext in ['*.jpg', '*.jpeg', '*.png']:
        image_paths.extend(list(Path(image_dir).glob(ext)))
    
    print(f"Found {len(image_paths)} images in the directory.")
    
    results = []
    for idx, image_path in enumerate(image_paths, 1):
        print(f"\nProcessing image {idx}/{len(image_paths)}: {image_path.name}")
        result = model.predict(source=str(image_path), save=True, device='cpu')
        results.append(result[0])
        
        # 각 이미지의 감지 결과 출력
        detected_classes = []
        confidences = []
        for box, cls in zip(result[0].boxes.conf, result[0].boxes.cls):
            class_name = result[0].names[int(cls)]
            confidence = float(box)
            detected_classes.append(class_name)
            confidences.append(confidence)
            print(f"- {class_name}: {confidence:.3f}")
        
        # 처리 속도 정보 출력
        preprocess_time = result[0].speed['preprocess']
        inference_time = result[0].speed['inference']
        postprocess_time = result[0].speed['postprocess']
        print(f"Processing times:")
        print(f"- Preprocess: {preprocess_time}ms")
        print(f"- Inference: {inference_time}ms")
        print(f"- Postprocess: {postprocess_time}ms")
        
    print(f"\nAll results saved to {output_dir}")
    return results

In [4]:
# Cell 4: JSON 라벨 변환 함수
def convert_labels_to_major_category(json_path, category_mapping):
    # JSON 파일 로드
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    converted_data = {}
    for image_name, annotations in data.items():
        converted_annotations = []
        for ann in annotations:
            class_name = ann["class_name"]
            if class_name in category_mapping:
                major_category = category_mapping[class_name]
                converted_annotations.append({
                    "class_name": major_category,
                    "bbox": ann["bbox"]
                })
        converted_data[image_name] = converted_annotations
    
    return converted_data


In [5]:
# Cell 5: 이진 분류 함수
def classify_images_by_major_category(converted_labels, target_class):
    binary_classification = {}
    
    for image_name, annotations in converted_labels.items():
        if any(ann["class_name"] == target_class for ann in annotations):
            binary_classification[image_name] = "Positive"
        else:
            binary_classification[image_name] = "Negative"
            
        # 상세 정보 출력
        detected_classes = [ann["class_name"] for ann in annotations]
        print(f"\nImage: {image_name}")
        print(f"Detected classes: {detected_classes}")
        print(f"Classification: {binary_classification[image_name]}")
    
    return binary_classification

In [6]:
# Cell 6: 결과 저장 함수
def save_results(binary_classification, output_path):
    results_summary = {
        "total_images": len(binary_classification),
        "positive_count": sum(1 for v in binary_classification.values() if v == "Positive"),
        "negative_count": sum(1 for v in binary_classification.values() if v == "Negative"),
        "classifications": binary_classification
    }
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(results_summary, f, indent=4, ensure_ascii=False)
    
    return results_summary

In [7]:
# Cell 7: Confidence Score 분석 함수
def calculate_confidence_scores(results):
    # 각 클래스별 confidence scores를 저장할 딕셔너리
    class_confidences = {
        'Large Waste Items': [],
        'PP bag': [],
        'General Waste': [],
        'CleanNet': []
    }
    
    # 모든 결과에 대해 순회
    for result in results:
        boxes = result.boxes
        for box, cls in zip(boxes.conf, boxes.cls):
            class_name = result.names[int(cls)]
            # 소분류를 대분류로 변환
            major_class = category_mapping.get(class_name, class_name)
            # confidence score 추가
            if major_class in class_confidences:
                class_confidences[major_class].append(float(box))
    
    # 각 클래스별 통계 계산
    statistics = {}
    for class_name, scores in class_confidences.items():
        if scores:  # 해당 클래스가 감지된 경우에만
            statistics[class_name] = {
                'mean': sum(scores) / len(scores),
                'max': max(scores),
                'min': min(scores),
                'count': len(scores),
                'all_scores': scores  # 모든 점수를 저장
            }
        else:
            statistics[class_name] = {
                'mean': 0,
                'max': 0,
                'min': 0,
                'count': 0,
                'all_scores': []
            }
    
    # 결과 출력
    print("\nConfidence Score Statistics by Class:")
    for class_name, stats in statistics.items():
        print(f"\n{class_name}:")
        print(f"- Number of detections: {stats['count']}")
        if stats['count'] > 0:
            print(f"- Average confidence: {stats['mean']:.3f}")
            print(f"- Maximum confidence: {stats['max']:.3f}")
            print(f"- Minimum confidence: {stats['min']:.3f}")
            # 모든 점수 출력
            print(f"- All confidence scores: {[f'{score:.3f}' for score in stats['all_scores']]}")
    
    return statistics

In [8]:
# Cell 8: 메인 실행 코드
def main():
    # 경로 설정
    model_path = r'E:\livingLab\YOLOv8기반 객체인식\results_20241124_155336\waste_detection_model\weights\best.pt'
    image_dir = r'E:\livingLab\YOLOv8기반 객체인식\test\images'
    json_path = r'E:\livingLab\YOLOv8기반 객체인식\test\test_annotations_yolo.json'
    target_class = 'Large Waste Items'
    
    # 1. 객체 감지 실행
    print("Starting object detection...")
    results = detect_objects(model_path, image_dir)
    
    # 2. 라벨 변환
    print("\nConverting labels to major categories...")
    converted_labels = convert_labels_to_major_category(json_path, category_mapping)
    
    # 3. 이진 분류 실행
    print("\nPerforming binary classification...")
    binary_classification = classify_images_by_major_category(converted_labels, target_class)
    
    # 4. 결과 저장
    output_path = './test/classification_results.json'
    results_summary = save_results(binary_classification, output_path)
    
    # 5. Confidence Score 분석
    print("\nAnalyzing confidence scores...")
    confidence_statistics = calculate_confidence_scores(results)
    
    # 6. Confidence Score 결과 저장
    confidence_output_path = './test/confidence_statistics.json'
    with open(confidence_output_path, 'w', encoding='utf-8') as f:
        json.dump(confidence_statistics, f, indent=4, ensure_ascii=False)
    
    # 7. 최종 결과 출력
    print("\nClassification Summary:")
    print(f"Total images processed: {results_summary['total_images']}")
    print(f"Positive classifications: {results_summary['positive_count']}")
    print(f"Negative classifications: {results_summary['negative_count']}")
    print(f"\nDetailed results saved to: {output_path}")
    print(f"Confidence statistics saved to: {confidence_output_path}")

In [9]:
# Cell 9: 스크립트 실행
if __name__ == "__main__":
    main()

Starting object detection...
Found 79 images in the directory.

Processing image 1/79: -7-_jpg.rf.3e4e45f01df8da1ac999a94efb60473e.jpg

image 1/1 E:\livingLab\YOLOv8 \test\images\-7-_jpg.rf.3e4e45f01df8da1ac999a94efb60473e.jpg: 640x640 1 PP bag, 50.3ms
Speed: 2.1ms preprocess, 50.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict10
- PP bag: 0.798
Processing times:
- Preprocess: 2.061128616333008ms
- Inference: 50.33278465270996ms
- Postprocess: 0.0ms

Processing image 2/79: 202411061313-36-307488-127-352668_jpg.rf.2debbe814df92503a0079be859650cc0.jpg

image 1/1 E:\livingLab\YOLOv8 \test\images\202411061313-36-307488-127-352668_jpg.rf.2debbe814df92503a0079be859650cc0.jpg: 640x640 1 General Waste, 2 CleanNets, 51.3ms
Speed: 0.0ms preprocess, 51.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict10
- CleanNet: 0.891
- CleanNet: 0.878
- General Waste: 0.690
Processing times:
- Prep